In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import datetime
%matplotlib inline
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import AdaBoostClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import xgboost as xgb
import pickle
from sklearn.tree import DecisionTreeClassifier 
from sklearn.metrics import accuracy_score, roc_curve, auc
from sklearn.model_selection import cross_val_score
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 225)
pickle_in=open("cleaned_data.pickle","rb")
df=pickle.load(pickle_in)
# I need to drop two columns that I left in for the visualization notebook
df.drop(['home_score','away_score'],axis=1,inplace=True)
# This is the same function from the previous notebook and it will be used to
# evaluate model performance
def calc_return(X_analyse):
    total_risk=[]
    total_reward=[]
    equal_bet_return=[]
    for i in range(len(X_analyse)):
        k=pd.DataFrame(X_analyse.iloc[i]).transpose()
        k.reset_index(drop=True,inplace=True)
        if int(k.preds[0])==1:
            if int(k.real[0])==1:
                if int(k.home_money[0])<0:
                    risk=k.home_money[0]
                    reward=100
                else:
                    risk=-100
                    reward=k.home_money[0]
            else:
                if k.home_money[0]<0:
                    risk=k.home_money[0]
                    reward=k.home_money[0]
                else:
                    risk=-100
                    reward=-100
        else:
            if int(k.real[0])==0:
                if k.away_money[0]<0:
                    risk=k.away_money[0]
                    reward=100
                else:
                    risk=-100
                    reward=k.away_money[0]
            else:
                if k.away_money[0]<0:
                    risk=k.away_money[0]
                    reward=k.away_money[0]
                else:
                    risk=-100
                    reward=-100
        total_risk.append(risk)
        total_reward.append(reward)
        equal_bet_winnings=reward/-risk*100
        equal_bet_return.append(equal_bet_winnings)
    natural_ror=round(-np.mean(total_reward)/np.mean(total_risk)*100,2)
    equal_bet_ror=round(np.mean(equal_bet_return),2)
    return natural_ror,equal_bet_ror
# This is a function for creating train-test splits that will work with my way of
# scoring model performance based on real world return on risk
def test_split(data,test_size,random_state):
    shuf_df=data.sample(frac=1,random_state=random_state)
    shuf_df.reset_index(drop=True,inplace=True)
    df2=shuf_df.copy()
    # This seperates the dataframe into data and target 
    X_temp=df2[df2.columns[1:]]
    y=df2.home_win
    # This standardized the data
    scaler=StandardScaler()
    X_s = scaler.fit_transform(X_temp)
    X=pd.DataFrame(X_s)
    # This does the train-test split in a way that I can carry through the odds values in order to calculate
    # the real-world usefulness of the model
    if len(X)==len(y):
        split_value=int(round(len(X)*(1-test_size),0))
        X_train=X.iloc[0:split_value]
        X_test=X.iloc[split_value:len(X)]
        y_train=y.iloc[0:split_value]
        y_test=y.iloc[split_value:len(X)]
        home_money=shuf_df.iloc[:,-2]
        away_money=shuf_df.iloc[:,-1]
    return X_train,X_test,y_train,y_test,home_money,away_money,split_value

Matplotlib is building the font cache; this may take a moment.


In [ ]:
results=pd.DataFrame([])
for crit in ['gini','entropy']:
    for depth in [3,10,20]:
        for samp_split in [10,25]:
            for num in [20,50,100]:
                holder=pd.DataFrame([])
                for j in range(0,10):
                    X_train,X_test,y_train,y_test,home_money,away_money,split_value=test_split(df,.2,j)
                    forest=RandomForestClassifier(n_estimators=num,criterion=crit,max_depth=depth,
                                                 min_samples_split=samp_split,random_state=j+10)
                    forest.fit(X_train,y_train)
                    train_pred=forest.predict(X_train)
                    test_pred=forest.predict(X_test)
                    bets=pd.DataFrame([])
                    for i in range(len(test_pred)):
                        if test_pred[i]>.5:
                            bets=bets.append(pd.DataFrame({'preds':1,'real':y_test[i+split_value],'home_money':home_money[i+split_value],
                                                     'away_money':away_money[i+split_value]},index=[0]),ignore_index=True)
                        else:
                            bets=bets.append(pd.DataFrame({'preds':0,'real':y_test[i+split_value],'home_money':home_money[i+split_value],
                                                     'away_money':away_money[i+split_value]},index=[0]),ignore_index=True)
                    train_acc=round(accuracy_score(y_train,train_pred)*100,1)
                    test_acc = round(accuracy_score(y_test,test_pred)*100,1)
                    nat,equal=calc_return(bets)
                    holder=holder.append(pd.DataFrame({'train':train_acc,'test':test_acc,
                                                        'nat':nat,'equal':equal},index=[0]),ignore_index=True)
                results=results.append(pd.DataFrame({'criteria':crit,'depth':depth,'samp_split':samp_split,
                                                    'num':num,'train':holder.train.mean(),
                                                     'test':holder.test.mean(),
                                                     'nat':holder.nat.mean(),
                                                     'equal':holder.equal.mean()},index=[0]),
                                       ignore_index=True)
print('Best Test Score:')
display(results[results.test==results.test.max()])
print('Best Natural Return on Risk:')
display(results[results.nat==results.nat.max()])
print('Best Equal Wager Return on Risk:')
display(results[results.equal==results.equal.max()])

In [ ]:
results=pd.DataFrame([])
for rate in [1,5,10,20]:
    for num in [20,50,100]:
        holder=pd.DataFrame([])
        for j in range(0,10):
            X_train,X_test,y_train,y_test,home_money,away_money,split_value=test_split(df,.2,j)
            ada=AdaBoostClassifier(learning_rate=rate,n_estimators=num)
            ada.fit(X_train,y_train)
            train_pred=ada.predict(X_train)
            test_pred=ada.predict(X_test)
            bets=pd.DataFrame([])
            for i in range(len(test_pred)):
                if test_pred[i]>.5:
                    bets=bets.append(pd.DataFrame({'preds':1,'real':y_test[i+split_value],'home_money':home_money[i+split_value],
                                             'away_money':away_money[i+split_value]},index=[0]),ignore_index=True)
                else:
                    bets=bets.append(pd.DataFrame({'preds':0,'real':y_test[i+split_value],'home_money':home_money[i+split_value],
                                             'away_money':away_money[i+split_value]},index=[0]),ignore_index=True)
            train_acc=round(accuracy_score(y_train,train_pred)*100,1)
            test_acc = round(accuracy_score(y_test,test_pred)*100,1)
            nat,equal=calc_return(bets)
            holder=holder.append(pd.DataFrame({'train':train_acc,'test':test_acc,
                                                'nat':nat,'equal':equal},index=[0]),ignore_index=True)
        results=results.append(pd.DataFrame({'learning_rate':rate,'num':num,
                                             'train':holder.train.mean(),
                                             'test':holder.test.mean(),
                                             'nat':holder.nat.mean(),
                                             'equal':holder.equal.mean()},index=[0]),
                               ignore_index=True)
print('Best Test Score:')
display(results[results.test==results.test.max()])
print('Best Natural Return on Risk:')
display(results[results.nat==results.nat.max()])
print('Best Equal Wager Return on Risk:')
display(results[results.equal==results.equal.max()])

In [ ]:
results=pd.DataFrame([])
for kern in ['linear','poly','sigmoid','rbf']:
    for c in [3,5,7,9]:
        print("c: " + str(c))
        for gam in ['auto',10,25]:
            print("gam: " + str(gam))
            holder=pd.DataFrame([])
            for j in range(0,10):
                X_train,X_test,y_train,y_test,home_money,away_money,split_value=test_split(df,.2,j)
                svm_clf=svm.SVC(kernel=kern,C=c,gamma=gam,random_state=j+10)
                svm_clf.fit(X_train,y_train)
                train_pred=svm_clf.predict(X_train)
                test_pred=svm_clf.predict(X_test)
                bets=pd.DataFrame([])
                for i in range(len(test_pred)):
                    if test_pred[i]>.5:
                        bets=bets.append(pd.DataFrame({'preds':1,'real':y_test[i+split_value],'home_money':home_money[i+split_value],
                                                 'away_money':away_money[i+split_value]},index=[0]),ignore_index=True)
                    else:
                        bets=bets.append(pd.DataFrame({'preds':0,'real':y_test[i+split_value],'home_money':home_money[i+split_value],
                                                 'away_money':away_money[i+split_value]},index=[0]),ignore_index=True)
                train_acc=round(accuracy_score(y_train,train_pred)*100,1)
                test_acc = round(accuracy_score(y_test,test_pred)*100,1)
                nat,equal=calc_return(bets)
                holder=holder.append(pd.DataFrame({'train':train_acc,'test':test_acc,
                                                    'nat':nat,'equal':equal},index=[0]),ignore_index=True)
            results=results.append(pd.DataFrame({'kernel':kern,'C':c,'gamma':gam,
                                                'train':holder.train.mean(),
                                                 'test':holder.test.mean(),
                                                 'nat':holder.nat.mean(),
                                                 'equal':holder.equal.mean()},index=[0]),
                                   ignore_index=True)
print('Best Test Score:')
display(results[results.test==results.test.max()])
print('Best Natural Return on Risk:')
display(results[results.nat==results.nat.max()])
print('Best Equal Wager Return on Risk:')
display(results[results.equal==results.equal.max()])

In [ ]:
# Note: Because there so many tunable parameters in the XG Boost model,
# I did a large amount of testing outside of this notebook and have only included
# a handful of potential values in order to save runtime
results=pd.DataFrame([])
for learn in [.01,.001]:
    print("learn:" + str(learn))
    for depth in [20,50]:
        for child in [3,10]:
            for num in [50,100,200]:
                for sub in [.4]:
                    for gam in [10]:
                        holder=pd.DataFrame([])
                        for j in range(0,10):
                            X_train,X_test,y_train,y_test,home_money,away_money,split_value=test_split(df,.2,j)
                            boost=xgb.XGBClassifier(learning_rate=learn,max_depth=depth,
                                                    min_child_weight=child,n_estimators=num,
                                                    subsample=sub,gamma=gam)
                            boost.fit(X_train,y_train)
                            train_pred=boost.predict(X_train)
                            test_pred=boost.predict(X_test)
                            bets=pd.DataFrame([])
                            for i in range(len(test_pred)):
                                if test_pred[i]>.5:
                                    bets=bets.append(pd.DataFrame({'preds':1,'real':y_test[i+split_value],'home_money':home_money[i+split_value],
                                                             'away_money':away_money[i+split_value]},index=[0]),ignore_index=True)
                                else:
                                    bets=bets.append(pd.DataFrame({'preds':0,'real':y_test[i+split_value],'home_money':home_money[i+split_value],
                                                             'away_money':away_money[i+split_value]},index=[0]),ignore_index=True)
                            train_acc=round(accuracy_score(y_train,train_pred)*100,1)
                            test_acc = round(accuracy_score(y_test,test_pred)*100,1)
                            nat,equal=calc_return(bets)
                            holder=holder.append(pd.DataFrame({'train':train_acc,'test':test_acc,
                                                                'nat':nat,'equal':equal},index=[0]),ignore_index=True)
                        results=results.append(pd.DataFrame({'learning_rate':learn,'depth':depth,
                                                             'child':child,'num':num,'sub':sub,
                                                             'gamma':gam,'train':holder.train.mean(),
                                                             'test':holder.test.mean(),
                                                             'nat':holder.nat.mean(),
                                                             'equal':holder.equal.mean()},index=[0]),
                                               ignore_index=True)
print('Best Test Score:')
display(results[results.test==results.test.max()])
print('Best Natural Return on Risk:')
display(results[results.nat==results.nat.max()])
print('Best Equal Wager Return on Risk:')
display(results[results.equal==results.equal.max()])

learn:0.01
learn:0.001
